In [ ]:
import pandas as pd
import shutil
import os


In [ ]:
data_path = './assets/data.xlsx'
if not os.path.isfile(data_path):
  from google_drive_downloader import GoogleDriveDownloader as gdd

  data_path = './data.xlsx'
  gdd.download_file_from_google_drive(file_id='1psRf7Nyg_iZPX5fkGq19RGkr6M08jKAd',
                                      dest_path=data_path,
                                      overwrite=True)

In [ ]:
## loads excel
xls = pd.ExcelFile(data_path)
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
!pip install konlpy

In [ ]:
## Kkma sentence
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text


s = time.time()

training_data_copy_kkma = copy.deepcopy(training_data)

cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data.values[:, 2][:20])
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()

print(e - s)

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
help(fasttext.FastText)

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

In [ ]:
!head cooking.stackexchange.txt

In [ ]:
!wc cooking.stackexchange.txt

In [ ]:
!head -n 12404 cooking.stackexchange.txt > cooking.train

In [ ]:
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

In [ ]:
model = fasttext.train_supervised(input="cooking.train")

In [ ]:
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
model.predict("Why not put knives in the dishwasher?")

In [ ]:
model.test("cooking.valid")

In [ ]:
model.test("cooking.valid", k=5)

In [ ]:
model.predict("Why not put knives in the dishwasher?", k=5)

In [ ]:
with open('./cooking.stackexchange.txt', 'r') as f:
  lines = f.readlines()
lines

In [ ]:
len(lines)